<a href="https://colab.research.google.com/github/Frosk-Kristian/COMP6002-Group10-Models/blob/develop/COMP6002_Group_Project_ML_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP6002 Computer Science Project - Group 10
Utilising Machine Learning to detect DDoS attacks.

# Setup

In [3]:
import pandas as pd
print("Using Pandas version: {}".format(pd.__version__))

Using Pandas version: 1.5.3


In [4]:
from sklearn import __version__ as skl_ver
print("Using Sklearn version: {}".format(skl_ver))

Using Sklearn version: 1.2.2


## Import & Export Trained Models

In [5]:
# helper functions for exporting/importing models trained with Sklearn, do not attempt to use with Neural Network as the library used has its' own methods for exporting/importing
# all functions defined will prompt the user for confirmation, to allow for skipping the functions when running the notebook
import joblib
print("Using Joblib version: {}".format(joblib.__version__))

# Save a trained model to the provided filepath
def SaveSKL(model, model_fpath):
  """
  Exports a trained model via joblib.

  Parameters:
    model (object): model to be saved.
    model_fpath (string): file path that the model will be saved in.
  Returns:
    : no value returned.
  """
  to_save = ""
  while to_save.lower() not in ('y', 'n'):
    to_save = input("Do you wish to save the trained model? (y/n)\n")
    if to_save.lower() in 'y':
      print("Saving model to: {}".format(model_fpath))
      try:
        joblib.dump(model, model_fpath)
        print("SUCCESS: Model saved to {}".format(model_fpath))
      except:
        print("ERROR: An unknown error has occured when calling joblib.dump()!")
    else:
      if to_save.lower() in 'n':
        print("Did not save model.")
      else:
        print("Please only enter \'y\' to save model or \'n\' to skip saving.")

# Load a trained model from the provided filepath
def LoadSKL(model_fpath):
  """
  Import a trained model via joblib.

  Parameters:
    model_fpath (string): file path to the stored model.
  Returns:
    object: if a model is found and loaded correctly, returns an object.
    None: if no matching file is found or an error occurs during loading, returns None.
  """
  to_load = ""
  while to_load.lower() not in ('y', 'n'):
    to_load = input("Do you wish to import a trained model? {y/n)\n")
    if to_load.lower() in 'y':
      model = None
      print("Attempting to import model from: {}".format(model_fpath))
      try:
        model = joblib.load(model_fpath)
        print("SUCCESS: Model successfully imported.")
      except FileNotFoundError:
        print("ERROR: The file \'{}\' does not exist!".format(model_fpath))
        model = None
      except:
        print("ERROR: An unknown error has occured when calling joblib.load()!")
        model = None
      finally:
        return model
    else:
      if to_load.lower() in 'n':
        print("Did not import model.")
        return None
      else:
        print("Please only enter \'y\' to import model or \'n\' to skip importing.")

# To-do: write function that exports model parameters, evaluation metrics, etc.

Using Joblib version: 1.3.2


# Import Data
Checks current working directory for datasets, if datasets are missing downloads a [.zip archive mirror of the CiCDDoS2019 hosted on Kaggle](https://www.kaggle.com/datasets/kristianfrossos/cicddos2019/data).

**NOTE:** the first part of this section is specific to Google Colab, and will not work outside of it. Advise writing an alternative later for local use (relevant when training Neural Network for speed and when usage limits get in the way).

## Reference
Iman Sharafaldin, Arash Habibi Lashkari, Saqib Hakak, and Ali A. Ghorbani, "Developing Realistic Distributed Denial of Service (DDoS) Attack Dataset and Taxonomy", IEEE 53rd International Carnahan Conference on Security Technology, Chennai, India, 2019.


In [6]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [7]:
# Sets up dataset directory
import os

In [8]:
# mounts google drive
from google.colab import drive
drive.mount("/content/drive")

# directory that the dataset will be downloaded to
dl_dir = os.getcwd() + r'/drive/MyDrive/Colab Notebooks/COMP6002_Group10_Data'

if os.path.exists(dl_dir):
  print("Directory {} already exists.\n".format(dl_dir))
else:
  os.mkdir(dl_dir)
  print("Successfully created the directory {}".format(dl_dir))

Mounted at /content/drive
Directory /content/drive/MyDrive/Colab Notebooks/COMP6002_Group10_Data already exists.



In [9]:
# sets up kaggle environment variables (needed to access API)
from google.colab import userdata
from google.colab import files

# checks if kaggle key and username have been provided as secrets and sets environment variables appropriately
# if not found, attempts to use kaggle.json
try:
  os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
  os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
  print("Using KAGGLE_KEY and KAGGLE_USERNAME defined in secrets.")
except (userdata.SecretNotFoundError, userdata.NotebookAccessError):
  print("WARN: One or more secret(s) missing or inaccessible.\n")
  if os.path.isfile('~/.kaggle/kaggle.json'):
    print("Using existing kaggle.json")
  else:
    print("Please upload kaggle.json")
    files.upload()

    if os.path.isfile(os.getcwd() + '/content/kaggle.json'):
      !rm -r ~/.kaggle
      !mkdir ~/.kaggle
      !mv ./kaggle.json ~/.kaggle/
      !chmod 600 ~/.kaggle/kaggle.json
    else:
      print("\'kaggle.json\' not uploaded.")
      raise

Using KAGGLE_KEY and KAGGLE_USERNAME defined in secrets.


In [10]:
data_dir = os.getcwd() + '/dataset'

# checks if .zip archive containing dataset already exists in google drive and downloads it if necessary
if os.path.isfile(dl_dir + '/cicddos2019.zip'):
  print("Dataset already present.")
else:
  print("Downloading zipped dataset to {}".format(dl_dir))
  !kaggle datasets download kristianfrossos/cicddos2019 -p {dl_dir.replace(' ', '\ ')}

# creates the content/dataset directory if it doesn't already exist
if os.path.exists(data_dir):
  print("Directory {} already exists.\n".format(data_dir))
else:
  print("Created directory: {}\n".format(data_dir))
  os.mkdir(data_dir)

# extracts contents of .zip archive to content/dataset if directory is not empty
if not os.listdir(data_dir):
  print("Empty directory, extracting dataset.")
  # unzips .zip archive
  !unzip {dl_dir.replace(' ', '\ ') + '/cicddos2019.zip'} -d {data_dir}
else:
  print("Non-empty directory, skipping download")

Dataset already present.
Created directory: /content/dataset

Empty directory, downloading dataset.
Archive:  /content/drive/MyDrive/Colab Notebooks/COMP6002_Group10_Data/cicddos2019.zip
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_DNS.csv  
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_LDAP.csv  
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_MSSQL.csv  
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_NTP.csv  
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_NetBIOS.csv  
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_SNMP.csv  
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_SSDP.csv  
  inflating: /content/dataset/CSV-01-12/01-12/DrDoS_UDP.csv  
  inflating: /content/dataset/CSV-01-12/01-12/Syn.csv  
  inflating: /content/dataset/CSV-01-12/01-12/TFTP.csv  
  inflating: /content/dataset/CSV-01-12/01-12/UDPLag.csv  
  inflating: /content/dataset/CSV-03-11/03-11/LDAP.csv  
  inflating: /content/dataset/CSV-03-11/03-11/MSSQL.csv  
  inflating: /content

In [37]:
# initialises empty list
csv_list = []

# iterates through all subdirectories of /content/dataset and appends the filepath of each .csv to csv_list
for root, dirs, files in os.walk(data_dir):
  for f in files:
    if f.endswith(".csv"):
      csv_list.append(os.path.join(root, f))

# if .csv files were found, displays number of files and prints each path
if not csv_list:
  print("No .csv files found!")
else:
  print("{} .csv founds found.".format(len(csv_list)))
  for csv in csv_list:
    print(csv)

18 .csv founds found.
/content/dataset/CSV-01-12/01-12/DrDoS_NetBIOS.csv
/content/dataset/CSV-01-12/01-12/DrDoS_NTP.csv
/content/dataset/CSV-01-12/01-12/DrDoS_MSSQL.csv
/content/dataset/CSV-01-12/01-12/TFTP.csv
/content/dataset/CSV-01-12/01-12/DrDoS_SSDP.csv
/content/dataset/CSV-01-12/01-12/DrDoS_UDP.csv
/content/dataset/CSV-01-12/01-12/Syn.csv
/content/dataset/CSV-01-12/01-12/DrDoS_SNMP.csv
/content/dataset/CSV-01-12/01-12/DrDoS_DNS.csv
/content/dataset/CSV-01-12/01-12/UDPLag.csv
/content/dataset/CSV-01-12/01-12/DrDoS_LDAP.csv
/content/dataset/CSV-03-11/03-11/NetBIOS.csv
/content/dataset/CSV-03-11/03-11/Portmap.csv
/content/dataset/CSV-03-11/03-11/LDAP.csv
/content/dataset/CSV-03-11/03-11/UDP.csv
/content/dataset/CSV-03-11/03-11/MSSQL.csv
/content/dataset/CSV-03-11/03-11/Syn.csv
/content/dataset/CSV-03-11/03-11/UDPLag.csv


In [1]:
# COMMENTED OUT AS THIS WILL CRASH GOOGLE COLAB (AND POTENTIALLY YOUR COMPUTER IF RUNNING LOCALLY) - not enough RAM to load whole dataset
# TO-DO:
# look at using memory-mapped arrays (numpy might have an implementation of this)
# look at using the library Dask

# df = pd.concat((pd.read_csv(csv) for csv in csv_list), ignore_index = True)

NameError: name 'pd' is not defined

## Data Preprocessing
**To-Do:**
*   Split dataset into x and y train, test and validation.
*   Normalise dataset and perform other transformations as required.

In [ ]:
# split dataset into separatate training and testing datasets
from sklearn.model_selection import train_test_split


# Random Forest

In [ ]:
# build random forest classifier model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(random_state = 42)

# grid of parameters to search through when performing cross validation
rf_params = {
    'class_weight' : ['balanced']
}

# tests all permutations of the parameters outline in rf_params, returns the best performing model
rf_model = GridSearchCV(estimator = rf,
                        param_grid = rf_params,
                        scoring = ["accuracy", "f1_weighted", "roc_auc_ovr"],
                        refit = "f1_weighted",
                        cv = 5,
                        verbose = 3,
                        return_train_score = True)

# rf_model.fit(x_train, y_train)

# Neural Network

## NN Setup
Import and install required libraries, sets some initial values.

In [ ]:
# import PyTorch and confirm version
import torch
from torch import nn
print("Using PyTorch version: {}".format(torch.__version__))

# check the availability of and set the device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print("Using {} device.".format(device))

Using PyTorch version: 2.2.1+cu121

Using cpu device.


In [ ]:
# install Skorch, providing a wrapper for using PyTorch with Sklearn
!pip install skorch

# import Skorch and confirm version
from skorch import __version__ as skorch_version
from skorch import NeuralNetClassifier
print("Using Skorch version: {}".format(skorch_version))

## Define Neural Network
Currently using a Multilayer Perceptron (MLP), consider swapping to a hybrid model of a MLP and Convolutional Neural Network (CNN) later.

In [ ]:
class NN_MLP(nn.Module):
  """Class that defines a multilayer perceptron model."""
  def __init__(self, input_size, hidden_size, output_size):
    """
    Construct a new NN_MLP object.

    Parameters:
      input_size (int): number of inputs to the input layer.
      hidden_size (int): number of inputs to the hidden layer(s).
      output_size (int): number of outputs from the output layer, typically 1.
    Returns:
      : no value returned.
    """
    super(NN_MLP, self).__init__()
    # layers
    self.h1 = nn.Linear(input_size, hidden_size)
    self.h2 = nn.Linear(hidden_size, hidden_size)
    self.output = nn.Linear(hidden_size, output_size)
    # activation functions
    self.relu = nn.ReLu()

  def forward(self, X):
    """
    Parameters:
      X (Any): features to make prediction on.
    Returns:
      Any: predicted value.
    """
    out = self.h1(X)
    out = self.relu(out)
    out = self.h2(out)
    out = self.relu(out)
    out = self.output(out)

    return out